# Testing the Efficacy of Airbnb's 90 Day Limit on Entire Home Listings

**Research Question:** Did the imposition of the 90 day limit reduce the growth rate in proportion of entire home listings over 90 days in comparison to single rooms?

**H0:** there is no difference in the change in proportion of entire home listings exceeding 90 days compared to single room listings after the imposition of the 90-day limit.

**HA:** there is a difference in the change in proportion of entire home listings exceeding 90 days compared to single room listings after the imposition of the 90-day limit.

The policy was introduced in Jan 2017, so I will test for differences in proportions between 2016 and 2017. There is, however, a possibility of a time lag, which means different years may yield different results.

As this notebook aims to be reproducible and we are getting close to submission time, I'm not going to keep record of any QAing/data exploration - I'll only keep things which I think might make it into the final document.

In [133]:
#Loading packages
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
import os
import datetime as dt
import seaborn as sns
import duckdb as db
import statsmodels.api as sm
from requests import get
from urllib.parse import urlparse
from functools import wraps
from scipy.stats import chi2_contingency
from scipy.stats import ttest_rel
import statsmodels.formula.api as smf
print(os.getcwd())

/home/jovyan/work/Documentation


## 1. Data Processing

### 1.1 Review Data

In [134]:
#Our Github has run out of storage so we cannot upload the data here
#Instead I will use Jon's code to download the June 2024 reviews from his website and save it locally

url  = 'https://orca.casa.ucl.ac.uk/~jreades/data/20240614-London-reviews.csv.gz'

def check_cache(f):
    @wraps(f)
    def wrapper(src, dst, min_size=100):
        url = urlparse(src) # We assume that this is some kind of valid URL 
        fn  = os.path.split(url.path)[-1] # Extract the filename
        dsn = os.path.join(dst,fn) # Destination filename
        if os.path.isfile(dsn) and os.path.getsize(dsn) > min_size:
            print(f"+ {dsn} found locally!")
            return(dsn)
        else:
            print(f"+ {dsn} not found, downloading!")
            return(f(src, dsn))
    return wrapper

@check_cache
def cache_data(src:str, dst:str) -> str:
    """Downloads a remote file.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dst : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """

    # Convert the path back into a list (without)
    # the filename -- we need to check that directories
    # exist first.
    path = os.path.split(dst)[0]
    print(f"Path: {path}")
      
    # Create any missing directories in dest(ination) path
    # -- os.path.join is the reverse of split (as you saw above)
    # but it doesn't work with lists... so I had to google how
    # to use the 'splat' operator! os.makedirs creates missing
    # directories in a path automatically.
    if path != '':
        os.makedirs(path, exist_ok=True)
        
    # Download and write the file
    with open(dst, "wb") as file:
        response = get(src)
        file.write(response.content)
        
    print(' + Done downloading...')

    return dst

ddir = os.path.join('Documentation','data', 'raw') # destination directory
path  = cache_data(url, ddir)
reviews = pd.read_csv(path, compression='gzip')
listings = pd.read_csv("data/raw/listings.csv.gz")
listings = listings[["id", "host_id", "room_type", "minimum_nights"]]

+ Documentation/data/raw/20240614-London-reviews.csv.gz found locally!


In [136]:
#Changing date data type
reviews["date"] = pd.to_datetime(reviews["date"], format="%Y-%m-%d")

#Splitting dates for last 24 months
print(f"Latest review date: {reviews.date.max()}.")
max_date = reviews.date.max()
cutoff_2023 = max_date.replace(year=max_date.year - 1)
cutoff_date = max_date.replace(year=max_date.year - 2)
print(f"This means the cutoff date for reviews is: {cutoff_date}.")

#Filter for only reviews from the last 24 months
#Update: changing this to include all reviews from 2022 onwards (if we are doing 2022/2023 rather than 12 month series)
reviews = reviews[reviews["date"] >= '01-01-2022']
min_date = reviews.date.min()- dt.timedelta(days=1)

#Add column with year data (ChatGPT helped)
reviews["year_category"] = pd.cut(reviews["date"],
                        bins=[min_date, cutoff_date, cutoff_2023, max_date],
                        labels=["Pre-Cutoff", "2022_2023", "2023_2024"],
                        right=True)

Latest review date: 2024-06-17 00:00:00.
This means the cutoff date for reviews is: 2022-06-17 00:00:00.


### 1.3 Join Tables and Calculate Occupancy Metric

Using Wang et al. (2024) occupancy estimation:
1. Count reviews per listing per year
2. Divide by 0.5 (assume that 1 in 2 stays results in a review)
3. Join to the listings dataset
4. Estimate stay length: either 3 days or minimum nights, whichever is larger
5. Multiply this by review rate
6. Cap at 21 nights per month (252) - *although technically 2016 was a leap year*

Finally, work out whether a listing had estimated over 90 nights or not

In [137]:
def over90proportions(reviews,listings,year_columns):
    #Step 1: count reviews per listing per year
    reviews_annual = reviews.groupby(['listing_id', 'year_category']).size().unstack(fill_value=0)
    reviews_annual = reviews_annual.reset_index()
    reviews_annual.columns.name = None
    
    #Step 2: divide each year by 0.5 (assume that 1 in 2 stays results in a review)
    for year in year_columns:
        reviews_annual[f'{year}_adjusted'] = reviews_annual[year] / 0.5
        
    #Step 3: join to the listings dataset
    reviews_annual = reviews_annual.merge(listings, how='left', left_on='listing_id', right_on='id').drop(columns=['id'])

    reviews_annual.dropna(subset=['room_type'], inplace=True)

    #Step 4: calculating estimated nights column: greater of either 3 or minimum nights
    reviews_annual['estimated_stay'] = np.maximum(3, reviews_annual.minimum_nights)

    #Step 5: estimate occupied nights for each year by multiplying the adjusted review rate by the estimated stay duration
    #n.b. this assumes that the minimum nights has not changed over time
    #Step 6: cap at 21 days per month (not changing 2016 leap year as 1/365 = 0.002...)
    
    cap_nights = 12 * 21  # max 21 days per month
    for year in year_columns:
        reviews_annual[f'est_nights_{year}'] = reviews_annual[f'{year}_adjusted'] * reviews_annual.estimated_stay
        reviews_annual[f'est_nights_{year}_capped'] = np.minimum(cap_nights, reviews_annual[f'est_nights_{year}'])

    return reviews_annual

In [139]:
reviews_annual = over90proportions(reviews,listings,['2022_2023','2023_2024'])

reviews_annual.head()

/tmp/ipykernel_13149/1669582155.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  reviews_annual = reviews.groupby(['listing_id', 'year_category']).size().unstack(fill_value=0)


### 1.2 Listing Data

In [141]:
#Calculating final table: whether a listing had True or False for over 90 days, and aggregated by room type
#Only looking at room type for now connected to research question, but I have the host column in there to check for superhosts if necessary

#Getting number of over 90s and totals for each category, as this is what the statistical test requires

#ChatGPT suggested taking a dynamic approach which integrates the year variable into the query and iterate over it
#But this felt a bit dishonest as it's not something I would have written myself
#So please excuse the slightly unwiedly query - it's not the most scalable but it is legible and it was written entirely by me!

db.register('reviews_annual', reviews_annual)

query = '''
       WITH listings_90 AS (
            SELECT 
            listing_id,
            CASE WHEN room_type = 'Entire home/apt' THEN 'Entire Home' ELSE 'Other' END AS room_type,
            CASE WHEN est_nights_2022_2023_capped >= 90 THEN 1 ELSE 0 END AS over90_2223,
            CASE WHEN est_nights_2022_2023_capped BETWEEN 1 AND 90 THEN 1 ELSE 0 END AS under90_2223,
            CASE WHEN est_nights_2023_2024_capped >= 90 THEN 1 ELSE 0 END AS over90_2324,
            CASE WHEN est_nights_2023_2024_capped BETWEEN 1 AND 90 THEN 1 ELSE 0 END AS under90_2324
        FROM reviews_annual)
    SELECT
        room_type,
        SUM(over90_2324) AS over90_2324,
        SUM(over90_2324) + SUM(under90_2324) AS total_2324,
        SUM(over90_2223) AS over90_2223,
        SUM(over90_2223) + SUM(under90_2223) AS total_2223
    FROM listings_90
    GROUP BY 1
'''

proportions_room = db.sql(query).to_df()
proportions_room.head()

,room_type,over90_2324,total_2324,over90_2223,total_2223
0,Other,5032.0,14090.0,4257.0,10002.0
1,Entire Home,6874.0,28244.0,4814.0,17391.0


In [142]:
#Calculating final table: whether a listing had True or False for over 90 days, and aggregated by host type
#The below calculation is not ideal as a host might have become a Superhost later

#Getting number of over 90s and totals for each category, as this is what the statistical test requires

query2 = '''
       WITH host_type AS (
       SELECT 
           host_id,
           CASE WHEN COUNT(*)>1 THEN 'Multi-Listing Host' ELSE 'Single Property Host' END AS host_type           
       FROM reviews_annual
       GROUP BY 1),
       
       listings_90 AS (
            SELECT 
            r.listing_id,
            h.host_type,
            CASE WHEN r.est_nights_2022_2023_capped >= 90 THEN 1 ELSE 0 END AS over90_2223,
            CASE WHEN r.est_nights_2022_2023_capped BETWEEN 1 AND 90 THEN 1 ELSE 0 END AS under90_2223,
            CASE WHEN r.est_nights_2023_2024_capped >= 90 THEN 1 ELSE 0 END AS over90_2324,
            CASE WHEN r.est_nights_2023_2024_capped BETWEEN 1 AND 90 THEN 1 ELSE 0 END AS under90_2324,
        FROM reviews_annual r
            LEFT JOIN host_type h
                ON r.host_id=h.host_id)
        
    SELECT
        host_type,
        SUM(over90_2223) AS over90_2223,
        SUM(over90_2223) + SUM(under90_2223) AS total_2223,
        SUM(over90_2324) AS over90_2324,
        SUM(over90_2324) + SUM(under90_2324) AS total_2324
    FROM listings_90
    GROUP BY 1
'''

proportions_host = db.sql(query2).to_df()
proportions_host.head()

,host_type,over90_2223,total_2223,over90_2324,total_2324
0,Multi-Listing Host,5397.0,14989.0,7492.0,24225.0
1,Single Property Host,3674.0,12404.0,4414.0,18109.0
